In [208]:
import matplotlib.pyplot as plt
import torch
from torch import nn
from sklearn import metrics

import numpy as np
import pandas as pd
import re

Innetől a beolvasáshoz segédlet

In [235]:
df_losses = pd.read_csv('model_losses.csv')
# One column
# p30_u10 - configuration key: Percentile 30, Uniformity 10
# nature - model type: nature, urban or all
# all - dataset type: nature, urban or all
# 0.6424242854118347 - loss value for picture 0
# 15 more float values for 15 more pictures
df_actual = pd.read_csv('image_actual.csv')

In [236]:
for col in df_losses.columns:
    # get first row value for this specific column
    col_name = re.sub('\.\d$', '', col)
    first_row = df_losses.iloc[0][col]
    second_row = df_losses.iloc[1][col]
    new_column_name = f"{col_name}_{first_row}_{second_row}"
    # rename the column with the existing column header plus the first row of that column's data
    df_losses.rename(columns={col: new_column_name}, inplace=True)

# delete first row of data, now that it's merged into the header
df_losses.drop(index=[0,1], inplace=True)
df_losses.drop(columns=df_losses.columns[0], axis=1, inplace=True)
df_losses.reset_index(drop=True, inplace=True)
df_losses=df_losses.astype(float)
for col in df_actual.columns:
    # get first row value for this specific column
    col_name = re.sub('\.\d$', '', col)
    first_row = df_actual.iloc[0][col]
    new_column_name = f"{col_name}_{first_row}"
    # rename the column with the existing column header plus the first row of that column's data
    df_actual.rename(columns={col: new_column_name}, inplace=True)

# delete first row of data, now that it's merged into the header
df_actual.drop(index=[0], inplace=True)
df_actual.drop(columns=df_actual.columns[0], axis=1, inplace=True)
df_actual.reset_index(drop=True,inplace=True)
df_actual=df_losses.astype(float)

In [237]:
df_losses_true = pd.DataFrame(df_losses, index=df_losses.index, columns=df_losses.columns)
df_actual_true = pd.DataFrame(df_actual, index=df_actual.index, columns=df_actual.columns)

In [238]:
for col in df_losses_true.columns:
  df_losses_true[col].apply(lambda x: 0 if x > 0.5 else 1)
for col in df_actual_true.columns:
  df_actual_true[col].apply(lambda x: 0 if x > 0.5 else 1)

Beolvasás és átalakítás vége

In [ ]:
def predictions(cutoff= 0.7):
    cutoff = cutoff                              # decide on a cutoff limit
    y_pred1 = []
    y_pred2 = []
    y_true = []
    for y in range(10):
        y = np.random.rand()
        y_true.append(y)
        y_pred1.append(y - 0.1*np.random.rand())
        y_pred2.append(np.random.rand() + 0.1*np.random.rand())
    y_true = np.array(y_true)
    y_pred1 = np.array(y_pred1)
    y_pred2 = np.array(y_pred2)

    y_true_classes = np.zeros_like(y_true)    # initialise a matrix full with zeros
    y_true_classes[y_true > cutoff] = 1       # add a 1 if the cutoff was breached
    y_pred_classes1 = np.zeros_like(y_pred1)    # initialise a matrix full with zeros
    y_pred_classes1[y_pred1 > cutoff] = 1       # add a 1 if the cutoff was breached
    y_pred_classes2 = np.zeros_like(y_pred2)    # initialise a matrix full with zeros
    y_pred_classes2[y_pred2 > cutoff] = 1       # add a 1 if the cutoff was breached
    return y_true, y_true_classes, y_pred1, y_pred_classes1, y_pred2, y_pred_classes2

In [ ]:
y_true, y_true_classes, y_pred1, y_pred_classes1 = predictions()

In [ ]:
#Dice-folytonos
Dice_pred1 = 2* np.sum(np.multiply(y_pred1, y_true))/(np.sum(y_pred1)*np.sum(y_true))
Dice_pred2 = 2* np.sum(np.multiply(y_pred2, y_true))/(np.sum(y_pred2)*np.sum(y_true))
Dice_preds = 2* np.sum(np.multiply(y_pred1, y_pred2))/(np.sum(y_pred1)*np.sum(y_pred2))
Dice_preds, Dice_pred1, Dice_pred2

(0.1979647592662276, 0.2905487175419531, 0.1964408422637171)

In [ ]:
y_true_classes, y_pred_classes1, y_pred_classes2

(array([0., 1., 0., 0., 0., 0., 0., 0., 0., 1.]),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]),
 array([0., 0., 0., 1., 1., 0., 1., 1., 0., 0.]))

In [ ]:
tn, fp, fn, tp = metrics.confusion_matrix([0, 1, 0, 1], [1, 1, 1, 0]).ravel()
tn, fp, fn, tp = metrics.confusion_matrix([0, 1, 0, 0, 0, 0], [0, 1, 1, 1, 0, 0]).ravel()
(tn, fp, fn, tp)

(3, 2, 0, 1)

In [ ]:
metrics.confusion_matrix([0, 1, 0, 0, 0, 0], [0, 1, 1, 1, 0, 0])

array([[3, 2],
       [0, 1]])

In [ ]:
tp, fp, fn, tn  = metrics.confusion_matrix(y_true_classes, y_pred_classes1).ravel() #roc_curve(test, pred, drop_intermediate=False)
print(tp, fp, fn, tn)

8 0 1 1


In [ ]:
#Intersection over Union
IoU = tp / (tp+fn+fp)
IoU

0.8888888888888888

In [ ]:
#Sørensen–Dice
DSC = (2*tp) / (2*tp+fn+fp)
DSC

0.9411764705882353